<a href="https://colab.research.google.com/github/Ryu4824/code-states/blob/main/n214_discussion_6%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **N214 Logistic Regression**

## 오늘의 목표
- scikit-learn을 이용해 로지스틱 회귀 모델을 만들어 학습하고 해석할 수 있습니다.
- 분류 평가지표를 이해하고 사용할 수 있습니다.  


## **개념 Topic**
> 오늘은 분류 모델인 로지스틱 회귀(Logistic Regression) 모델에 대해 배웠습니다. 

- 로지스틱 회귀를 한 문장으로 요약하여 설명해보세요.
  - `로지스틱 회귀모델은 0과 1 사이의 값을 출력하며 출력값이 0.5 이상일 경우는 Class1로 0.5 미만일 경우는 Class0으로 분류합니다.`를 제외한 문장이어야 합니다.
  - 어떤 키워드가 포함되어야할지 키워드 위주로 고민해보세요.
  - 만약 설명이 어렵다면 어느 부분이 이해가 안 되는지 구체적인 질문을 동기들과 나눠보세요.
- **Discussion** 표의 `정리` 탭에 답변을 정리하여 적어 주세요. 

## **코딩 Topic**

### **Part.1 : 데이터 준비**
통신사 고객의 계약 해지 데이터셋인 [Telco Customer Churn](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)를 사용해서 해지 여부를 예측하는 분류 문제를 풀어보겠습니다.

데이터에 대한 더 자세한 설명은 위의 링크를 참고하세요.
> **Data Description**

- customerID : Customer ID
- gender : Whether the customer is a male or a female
- SeniorCitizen : Whether the customer is a senior citizen or not (1, 0)
- Partner : Whether the customer has a partner or not (Yes, No)
- Dependents : Whether the customer has dependents or not (Yes, No)
- tenure : Number of months the customer has stayed with the company
- PhoneService : Whether the customer has a phone service or not (Yes, No)
- MultipleLines : Whether the customer has multiple lines or not (Yes, No, No phone service)
- InternetService : Customer’s internet service provider (DSL, Fiber optic, No)
- OnlineSecurity : Whether the customer has online security or not (Yes, No, No internet service)
- OnlineBackup : Whether the customer has online backup or not (Yes, No, No internet service)
- DeviceProtection : Whether the customer has device protection or not (Yes, No, No internet service)
- TechSupport : Whether the customer has tech support or not (Yes, No, No internet service)
- StreamingTV : Whether the customer has streaming TV or not (Yes, No, No internet service)
- StreamingMovies : Whether the customer has streaming movies or not (Yes, No, No internet service)
- Contract : The contract term of the customer (Month-to-month, One year, Two year)
- PaperlessBilling : Whether the customer has paperless billing or not (Yes, No)
- PaymentMethod : The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))
- MonthlyCharges : The amount charged to the customer monthly
- TotalCharges : The total amount charged to the customer
- Churn : Whether the customer churned or not (Yes or No)

In [ ]:
#해당 라이브러리 설치 후 런타임 다시 시작 -> 이후 셀 진행
!pip install scikit-learn==1.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 23.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 2.2 MB/s eta 0:00:00


#### **1-1. 데이터 불러오기 및 전처리**
- 데이터셋을 불러오세요. 
- `customerID` colulmn을 제거하세요. 
- 결측치 및 중복인 샘플(row)을 제거하세요. 
- `TotalCharges` column에서 공란인 데이터는 모두 0으로 바꾼 후, `TotalCharges` 데이터 타입을 `float`으로 변환하세요. 
- 타겟 클래스 비율을 확인해 보세요. 
  - 타겟 변수는 `Churn`입니다. 

In [ ]:
# 데이터셋 불러오기 
import pandas as pd
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/renewal/mldl/Customer_Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
# customer ID column 제거
df.drop('customerID', axis = 1, inplace = True)

In [ ]:
# 결측값 확인
# df.isnull().sum()
# 중복값 확인(22개) 후 제거
# df.duplicated().sum()
df.drop_duplicates(inplace = True)
# index 초기화
df.reset_index(drop = True, inplace = True)

In [ ]:
# TotalCharges column 공란 0으로 변경
df['TotalCharges'] = df['TotalCharges'].replace({' ' : '0'})
# TotalCharges data type str to float
df['TotalCharges'] = df['TotalCharges'].astype('float')

In [ ]:
# Target class 비율 확인
# No : 73% / Yes : 27%
df['Churn'].value_counts(normalize = True)

No     0.73463
Yes    0.26537
Name: Churn, dtype: float64

In [ ]:
df['TotalCharges']

0         29.85
1        1889.5
2        108.15
3       1840.75
4        151.65
         ...   
7038     1990.5
7039     7362.9
7040     346.45
7041      306.6
7042     6844.5
Name: TotalCharges, Length: 7043, dtype: object

In [ ]:
df.TotalCharges[488]

' '

'29.85'

#### **1-2. 데이터셋 분리**
- 특성 X와 타겟 y를 지정하세요.
  - 타겟 데이터를 `No`는 0으로, `Yes`는 1로 변환하세요.
- X와 y 모두 train/test set으로 분리하세요.
  - train : test 비는 8:2로 설정하세요.
  - `random_state`를 설정하여 고정된 결과가 나오도록 해 보세요.
  - 타겟 클래스 비율이 유지되도록 분리하세요.
    - [공식문서](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)를 참고하여 어떤 옵션을 설정하면 되는지 찾아보세요. 

#### **1-3. Scaling 후 데이터를 확인합니다.**
- 수치형 변수에 `StandardScaler()`를 적용하세요.

#### **1-4. Encoding 후 데이터를 확인합니다.**
- 범주형 변수에 `OneHotEncoder()`를 적용하세요. 
  - [공식문서](https://contrib.scikit-learn.org/category_encoders/onehot.html)를 참고하여 필요한 파라미터들을 설정해 볼 수 있습니다. 

### **Part.2 : 모델 학습 및 예측**

#### **2-1. 기준모델**
- 최빈값을 이용하여 기준모델을 만들고, accuracy 값을 확인하세요. 
  - 기준모델은 train set에 대하여 생성합니다. 

#### **2-2. 로지스틱 회귀**
- 로지스틱 회귀 모델을 학습하고, train 및 test set에 대하여 accuracy 값을 확인하세요. 
  - 타겟 클래스 비율이 불균형하므로 `class_weight='balanced'`로 설정하세요. 
- test set에 `.predict()`와 `.predict_proba()`를 적용하여 결과를 비교해 보세요. 
  - 10개의 데이터만 확인해 보세요.

#### **2-3. 회귀계수 해석**
- 회귀계수가 양수인 상위 3개 특성을 확인해 보세요. 
- 회귀계수가 음수인 하위 3개 특성을 확인해 보세요. 

### **Part.3 : 성능 평가**

#### **3-1. Confusion Matrix**
- test set에 대한 예측값을 바탕으로 confusion matrix를 나타내 보세요.

#### **3-2. 분류 평가지표**
- test set에 대하여 precision, recall, f1 score 값을 확인하세요.

#### **3-3. AUC Score**
- test set에 대하여 AUC Score 값을 확인하세요. 

### **Conclusion**
> 오늘 Topic을 수행한 결과를 바탕으로, 다음 사항에 대해 발표해 주세요.

1. 회귀계수가 양수인 상위 3개 특성을 바탕으로 결과를 해석해 보세요. 
2. 회귀계수가 음수인 하위 3개 특성을 바탕으로 결과를 해석해 보세요. 
3. 기준모델과 로지스틱 회귀 모델의 accuracy 값을 비교해 보세요. 평가지표로 accuracy를 사용하기에 적절한가요? 
4. Confusion Matrix 및 적절한 분류 평가지표를 선택하여 결과를 해석해 보세요. 
- **Discussion** 표의 `정리` 탭에 답변을 정리하여 적어 주세요. 

## **심화 Topic**
- 주변에서 찾아볼 수 있는 분류 문제는 무엇이 있을까요? 여러분이 실제로 프로젝트를 한다고 생각하고 관심 도메인에서 구체적으로 어떤 예시가 있을지 생각해보세요.
  - 여러분들은 어떤 프로젝트가 하고 싶으세요? 서로 아이디어를 나눠보세요.
  - 해당 문제를 풀었을 때 비즈니스적으로 어떤 가치있는 결론을 도출할 수 있을지도 함께 생각해 보세요. 